In [1]:
import pandas as pd
import os
import time
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import time
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc


C:\Users\Javi\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import random

def replace_ambiguous_bases(sequence):
    def resolve_ambiguity(code):
        if code == 'R':
            return random.choice(['A', 'G'])
        elif code == 'Y':
            return random.choice(['C', 'T'])
        elif code == 'S':
            return random.choice(['G', 'C'])
        elif code == 'W':
            return random.choice(['A', 'T'])
        elif code == 'K':
            return random.choice(['G', 'T'])
        elif code == 'M':
            return random.choice(['A', 'C'])
        elif code == 'B':
            return random.choice(['C', 'G', 'T'])
        elif code == 'D':
            return random.choice(['A', 'G', 'T'])
        elif code == 'H':
            return random.choice(['A', 'C', 'T'])
        elif code == 'V':
            return random.choice(['A', 'C', 'G'])
        elif code == 'N':
            return random.choice(['A', 'T', 'C', 'G'])
        else:
            return code  # If the code is not an ambiguity code, return the same code
    
    resolved_sequence = ''.join(resolve_ambiguity(base) for base in sequence)
    return resolved_sequence

In [3]:
def reverse_complement(sequence):
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    reverse_seq = sequence[::-1]  # Reverse the sequence
    reverse_complement_seq = ''.join(complement[base] for base in reverse_seq)
    return reverse_complement_seq

def reverse_complement_list(sequence_list):
    reverse_complements = []
    for seq in sequence_list:
        reverse_complements.append(reverse_complement(seq))
    return reverse_complements

In [4]:
#This function gets the cgr for the sequences
def easy_cgr(dna_sequence):
  nucleotide_map = {
    "A": (1, 1),
    "T": (-1, -1),
    "C": (1, -1),
    "G": (-1, 1),
}
  values=[]
  point=(0,0)
  for nucleotide in dna_sequence:
        next_point=nucleotide_map[nucleotide]
        point=((point[0]+next_point[0])/2,(point[1]+next_point[1])/2)
        values.append(point)
  return values

In [5]:
#This function has input:
#sequence to plot that is the easy_cgr(sequence)
# plot id that is the name
# resolution that is the resolution
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

def plotting_cgr(sequence_to_plot,id_to_plot,resolution):

  # Define the resolution
  num_bins = resolution

  # Create a grid for the histogram
  xedges = np.linspace(-1, 1, num_bins + 1)
  yedges = np.linspace(-1, 1, num_bins + 1)

  # Create a 2D histogram
  hist, _, _ = np.histogram2d([p[0] for p in sequence_to_plot], [p[1] for p in sequence_to_plot], bins=[xedges, yedges])

  # Create a custom colormap spanning from light gray to dark gray
  colors = [(1, 1, 1), (0.2, 0.2, 0.2)]  # Light gray to dark gray
  cmap_name = "custom_gray"
  custom_cmap = LinearSegmentedColormap.from_list(cmap_name, colors, N=len(sequence_to_plot) + 1)

  # Create a figure
  plt.figure()

  # Display the 2D histogram with the custom colormap
  plt.imshow(hist.T, cmap=custom_cmap, extent=[-1, 1, -1, 1], origin='lower', interpolation='nearest', aspect='auto')

  # Calculate coordinates for labels
  plt.xticks([])
  plt.yticks([])
  x_margin, y_margin = 0.05, 0.05
  label_coordinates = {
      "A": (1+ x_margin, 1 +y_margin),
      "T": (-1 - x_margin, -1-y_margin),
      "C": (1 + x_margin, -1 -y_margin),
      "G": (-1 - x_margin, 1 + y_margin),
  }

  # Add labels outside the grid
  for letter, (x, y) in label_coordinates.items():
      plt.text(x, y, letter, ha='center', va='center', fontsize=15, color='black')
  # Show the plot
  plt.title('CGR for '+id_to_plot)
  plt.grid(False)  # Turn off grid lines
  plt.show()

In [6]:
#This function reads the file when usingantibiotic resistance genes
import re

def read_antibiotic_resistance_genes(fasta_file_path):
    # Initialize lists to store sequence data
    sequence_ids = []
    sequences = []
    terms_inside_brackets = []  # Initialize a list to store terms inside brackets

    # Define a regular expression pattern to match text inside square brackets
    pattern = r'\[([^]]+)\]'

    # Open the FASTA file for reading
    with open(fasta_file_path, "r") as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespace

            # Check if the line is an ID line (starts with '>')
            if line.startswith(">"):
                # If it's not the first sequence, process the previous one
                if sequence_ids:
                    term_inside_brackets = re.search(pattern, sequence_ids[-1]).group(1)
                    terms_inside_brackets.append(term_inside_brackets)  # Append to the list
                # Extract the sequence ID from the ID line
                sequence_ids.append(line[1:])
                sequences.append("")  # Reset the sequence
            else:
                # Append the line to the sequence
                sequences[-1] += line

    # Process the last sequence in the file
    if sequence_ids:
        term_inside_brackets = re.search(pattern, sequence_ids[-1]).group(1)
        terms_inside_brackets.append(term_inside_brackets)

    return sequences, terms_inside_brackets

In [7]:
from Bio import SeqIO

def read_fasta_patients(file_path):
    sequence_ids = []
    sequences = []
    
    try:
        with open(file_path, "r") as fasta_file:
            fasta_sequences = SeqIO.parse(fasta_file, "fasta")
            
            for seq_record in fasta_sequences:
                sequence_ids.append(seq_record.id)
                sequences.append(str(seq_record.seq))
    except FileNotFoundError:
        print("File not found. Please provide a valid file path.")
    
    return sequences, sequence_ids


In [8]:
#This function has input:
#sequence to plot that is the easy_cgr(sequence)
# plot id that is the name
# resolution that is the resolution
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

def plotting_cgr(sequence_to_plot,id_to_plot,resolution):

  # Define the resolution
  num_bins = resolution

  # Create a grid for the histogram
  xedges = np.linspace(-1, 1, num_bins + 1)
  yedges = np.linspace(-1, 1, num_bins + 1)

  # Create a 2D histogram
  hist, _, _ = np.histogram2d([p[0] for p in sequence_to_plot], [p[1] for p in sequence_to_plot], bins=[xedges, yedges])

  # Create a custom colormap spanning from light gray to dark gray
  colors = [(1, 1, 1), (0.2, 0.2, 0.2)]  # Light gray to dark gray
  cmap_name = "custom_gray"
  custom_cmap = LinearSegmentedColormap.from_list(cmap_name, colors, N=len(sequence_to_plot) + 1)

  # Create a figure
  plt.figure()

  # Display the 2D histogram with the custom colormap
  plt.imshow(hist.T, cmap=custom_cmap, extent=[-1, 1, -1, 1], origin='lower', interpolation='nearest', aspect='auto')

  # Calculate coordinates for labels
  plt.xticks([])
  plt.yticks([])
  x_margin, y_margin = 0.05, 0.05
  label_coordinates = {
      "A": (1+ x_margin, 1 +y_margin),
      "T": (-1 - x_margin, -1-y_margin),
      "C": (1 + x_margin, -1 -y_margin),
      "G": (-1 - x_margin, 1 + y_margin),
  }

  # Add labels outside the grid
  for letter, (x, y) in label_coordinates.items():
      plt.text(x, y, letter, ha='center', va='center', fontsize=15, color='black')
  # Show the plot
  plt.title('CGR for '+id_to_plot)
  plt.grid(False)  # Turn off grid lines
  plt.show()


In [9]:
import random

def replace_ambiguous_bases(sequence):
    def resolve_ambiguity(code):
        if code == 'R':
            return random.choice(['A', 'G'])
        elif code == 'Y':
            return random.choice(['C', 'T'])
        elif code == 'S':
            return random.choice(['G', 'C'])
        elif code == 'W':
            return random.choice(['A', 'T'])
        elif code == 'K':
            return random.choice(['G', 'T'])
        elif code == 'M':
            return random.choice(['A', 'C'])
        elif code == 'B':
            return random.choice(['C', 'G', 'T'])
        elif code == 'D':
            return random.choice(['A', 'G', 'T'])
        elif code == 'H':
            return random.choice(['A', 'C', 'T'])
        elif code == 'V':
            return random.choice(['A', 'C', 'G'])
        elif code == 'N':
            return random.choice(['A', 'T', 'C', 'G'])
        else:
            return code  # If the code is not an ambiguity code, return the same code
    
    resolved_sequence = ''.join(resolve_ambiguity(base) for base in sequence)
    return resolved_sequence

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

def generate_cgr_and_save(sequence_to_plot, kmeres, output_filename):
    # Define the resolution
    num_bins = 2 ** kmeres

    # Create a grid for the histogram
    xedges = np.linspace(-1, 1, num_bins + 1)
    yedges = np.linspace(-1, 1, num_bins + 1)

    # Create a 2D histogram
    hist, _, _ = np.histogram2d(
        [p[0] for p in sequence_to_plot],
        [p[1] for p in sequence_to_plot],
        bins=[xedges, yedges]
    )

    # Create a custom colormap spanning from light gray to dark gray
    colors = [(1, 1, 1), (0.2, 0.2, 0.2)]  # Light gray to dark gray
    cmap_name = "custom_gray"
    custom_cmap = LinearSegmentedColormap.from_list(cmap_name, colors, N=len(sequence_to_plot) + 1)

    # Create a square figure with the desired size in inches
    dpi = 100
    fig, ax = plt.subplots(figsize=(84 / dpi, 84 / dpi), dpi=dpi)

    # Plot the histogram using imshow and set the extent to specify the image size
    ax.imshow(
        hist.T,
        cmap=custom_cmap,
        origin='lower',
        extent=[-1, 1, -1, 1],  # Setting extent to create a 100x100 pixel image
        interpolation='nearest'  # Optional: adjust the interpolation method
    )

    # Turn off the axis
    ax.axis('off')

    # Save the figure directly as an image using matplotlib.pyplot.savefig
    plt.savefig(output_filename, bbox_inches='tight', pad_inches=0, dpi=dpi)
    plt.close()

# Your remaining function remains unchanged
def generate_and_save_cgr_images(sequences, ids, output_dir, kmeres):
    # Create the output directory if it doesn't exist
    

    for i, (sequence, label) in enumerate(zip(sequences, ids)):
        # Generate the image filename based on the class label and sequence index
        image_filename = os.path.join(output_dir, f"{i}_{label}.png")

        # Generate and save the CGR image
        generate_cgr_and_save(sequence, kmeres, image_filename)

In [11]:

def read_fasta_ARG(file_name):
    sequences = []
    with open(file_name, 'r') as file:
        sequence_id = ''
        sequence = ''
        for line in file:
            line = line.strip()
            if line.startswith('>'):
                if sequence_id != '':
                    sequences.append(sequence_id.split('|') + [sequence])
                sequence_id = line[1:]
                sequence = ''
            else:
                sequence += line
        if sequence_id != '':
            sequences.append(sequence_id.split('|') + [sequence])
    return sequences

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

def generate_cgr_and_save(sequence_to_plot, kmeres, output_filename):
    # Define the resolution
    num_bins = 2 ** kmeres

    # Create a grid for the histogram
    xedges = np.linspace(-1, 1, num_bins + 1)
    yedges = np.linspace(-1, 1, num_bins + 1)

    # Create a 2D histogram
    hist, _, _ = np.histogram2d(
        [p[0] for p in sequence_to_plot],
        [p[1] for p in sequence_to_plot],
        bins=[xedges, yedges]
    )

    # Create a custom colormap spanning from light gray to dark gray
    colors = [(1, 1, 1), (0.2, 0.2, 0.2)]  # Light gray to dark gray
    cmap_name = "custom_gray"
    custom_cmap = LinearSegmentedColormap.from_list(cmap_name, colors, N=len(sequence_to_plot) + 1)

    # Create a square figure with the desired size in inches
    dpi = 100
    fig, ax = plt.subplots(figsize=(100 / dpi, 100 / dpi), dpi=dpi)

    # Plot the histogram using imshow and set the extent to specify the image size
    ax.imshow(
        hist.T,
        cmap=custom_cmap,
        origin='lower',
        extent=[-1, 1, -1, 1],  # Setting extent to create a 100x100 pixel image
        interpolation='nearest'  # Optional: adjust the interpolation method
    )

    # Turn off the axis
    ax.axis('off')

    # Save the figure directly as an image using matplotlib.pyplot.savefig
    plt.savefig(output_filename, bbox_inches='tight', pad_inches=0, dpi=dpi)
    plt.close()

# Your remaining function remains unchanged
def generate_and_save_cgr_images_label(sequences, class_labels, output_dir, kmeres):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    for i, (sequence, label) in enumerate(zip(sequences, class_labels)):
        # Generate the image filename based on the class label and sequence index
        image_filename = os.path.join(output_dir, f"{label}.png")

        # Generate and save the CGR image
        generate_cgr_and_save(sequence, kmeres, image_filename)

# Load Images

## Patients

In [13]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define the paths to your dataset directories
dataset_dir_0 = "patient_1"

# Initialize empty lists to store images and labels
# Define image dimensions
img_height, img_width = 100, 100
images_patient_1 = []
ids_patient_1 = []  # List to store image labels
# Load and preprocess images from directory 0 (label 0)
image_files = os.listdir(dataset_dir_0)
print("Number of image files in the directory for label 0:", len(image_files))

for image_file in os.listdir(dataset_dir_0):
    # Load and preprocess the image
    img = load_img(os.path.join(dataset_dir_0, image_file), target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    # Append the image to the images list
    images_patient_1.append(img_array)
    # Extract the label from the image filename
    label = os.path.splitext(image_file)[0]  # Remove file extension
    ids_patient_1.append(label)

# Now labels_patient_1 contains the labels for the images


Number of image files in the directory for label 0: 10981


In [14]:
len(images_patient_1)

10981

In [15]:

# Define the paths to your dataset directories
dataset_dir_0 = "patient_2"

# Initialize empty lists to store images and labels
# Define image dimensions
img_height, img_width = 100, 100
images_patient_2 = []
ids_patient_2 = []  # List to store image names
# Load and preprocess images from directory 0 (label 0)
image_files = os.listdir(dataset_dir_0)
print("Number of image files in the directory for label 0:", len(image_files))

for image_file in os.listdir(dataset_dir_0):
    # Load and preprocess the image
    img = load_img(os.path.join(dataset_dir_0, image_file), target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    # Append the image to the images list
    images_patient_2.append(img_array)
    # Extract the label from the image filename
    label = os.path.splitext(image_file)[0]  # Remove file extension
    ids_patient_2.append(label)

    # Assuming you also have a list called labels, you need to define it somewhere in your code
    # labels.append(0)

# Now ids_patient_1 contains the names of the images


Number of image files in the directory for label 0: 15000


In [16]:
len(images_patient_2)

15000

In [17]:

# Define the paths to your dataset directories
dataset_dir_0 = "patient_4"

# Initialize empty lists to store images and labels
# Define image dimensions
img_height, img_width = 100, 100
images_patient_4 = []
ids_patient_4 = []  # List to store image names
# Load and preprocess images from directory 0 (label 0)
image_files = os.listdir(dataset_dir_0)
print("Number of image files in the directory for label 0:", len(image_files))

for image_file in os.listdir(dataset_dir_0):
    # Load and preprocess the image
    img = load_img(os.path.join(dataset_dir_0, image_file), target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    # Append the image to the images list
    images_patient_4.append(img_array)
    # Extract the label from the image filename
    label = os.path.splitext(image_file)[0]  # Remove file extension
    ids_patient_4.append(label)
    # Assuming you also have a list called labels, you need to define it somewhere in your code
    # labels.append(0)

# Now ids_patient_1 contains the names of the images


Number of image files in the directory for label 0: 9172


In [18]:
len(images_patient_4)

9172

In [19]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define the paths to your dataset directories
dataset_dir_0 = "patient_5"

# Initialize empty lists to store images and labels
# Define image dimensions
img_height, img_width = 100, 100
images_patient_5 = []
ids_patient_5 = []  # List to store image labels
# Load and preprocess images from directory 0 (label 0)
image_files = os.listdir(dataset_dir_0)
print("Number of image files in the directory for label 0:", len(image_files))

for image_file in os.listdir(dataset_dir_0):
    # Load and preprocess the image
    img = load_img(os.path.join(dataset_dir_0, image_file), target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    # Append the image to the images list
    images_patient_5.append(img_array)
    # Extract the label from the image filename
    label = os.path.splitext(image_file)[0]  # Remove file extension
    ids_patient_5.append(label)

# Now labels_patient_1 contains the labels for the images


Number of image files in the directory for label 0: 15000


In [20]:
len(images_patient_5)

15000

In [21]:

# Define the paths to your dataset directories
dataset_dir_0 = "patient_6"

# Initialize empty lists to store images and labels
# Define image dimensions
img_height, img_width = 100, 100
images_patient_6 = []
ids_patient_6 = []  # List to store image names
# Load and preprocess images from directory 0 (label 0)
image_files = os.listdir(dataset_dir_0)
print("Number of image files in the directory for label 0:", len(image_files))

for image_file in os.listdir(dataset_dir_0):
    # Load and preprocess the image
    img = load_img(os.path.join(dataset_dir_0, image_file), target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    # Append the image to the images list
    images_patient_6.append(img_array)
    # Extract the label from the image filename
    label = os.path.splitext(image_file)[0]  # Remove file extension
    ids_patient_6.append(label)

    # Assuming you also have a list called labels, you need to define it somewhere in your code
    # labels.append(0)

# Now ids_patient_1 contains the names of the images


Number of image files in the directory for label 0: 10267


In [22]:
len(images_patient_6)

10267

In [23]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define the paths to your dataset directories
dataset_dir_0 = "patient_11"

# Initialize empty lists to store images and labels
# Define image dimensions
img_height, img_width = 100, 100
images_patient_11 = []
ids_patient_11 = []  # List to store image labels
# Load and preprocess images from directory 0 (label 0)
image_files = os.listdir(dataset_dir_0)
print("Number of image files in the directory for label 0:", len(image_files))

for image_file in os.listdir(dataset_dir_0):
    # Load and preprocess the image
    img = load_img(os.path.join(dataset_dir_0, image_file), target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    # Append the image to the images list
    images_patient_11.append(img_array)
    # Extract the label from the image filename
    label = os.path.splitext(image_file)[0]  # Remove file extension
    ids_patient_11.append(label)

# Now labels_patient_1 contains the labels for the images


Number of image files in the directory for label 0: 10022


In [24]:
len(images_patient_11)

10022

In [25]:

# Define the paths to your dataset directories
dataset_dir_0 = "patient_12"

# Initialize empty lists to store images and labels
# Define image dimensions
img_height, img_width = 100, 100
images_patient_12 = []
ids_patient_12 = []  # List to store image names
# Load and preprocess images from directory 0 (label 0)
image_files = os.listdir(dataset_dir_0)
print("Number of image files in the directory for label 0:", len(image_files))

for image_file in os.listdir(dataset_dir_0):
    # Load and preprocess the image
    img = load_img(os.path.join(dataset_dir_0, image_file), target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    # Append the image to the images list
    images_patient_12.append(img_array)
    # Extract the label from the image filename
    label = os.path.splitext(image_file)[0]  # Remove file extension
    ids_patient_12.append(label)

    # Assuming you also have a list called labels, you need to define it somewhere in your code
    # labels.append(0)

# Now ids_patient_1 contains the names of the images


Number of image files in the directory for label 0: 8445


In [26]:
len(images_patient_12)

8445

In [27]:

# Define the paths to your dataset directories
dataset_dir_0 = "patient_13"

# Initialize empty lists to store images and labels
# Define image dimensions
img_height, img_width = 100, 100
images_patient_13 = []
ids_patient_13 = []  # List to store image names
# Load and preprocess images from directory 0 (label 0)
image_files = os.listdir(dataset_dir_0)
print("Number of image files in the directory for label 0:", len(image_files))

for image_file in os.listdir(dataset_dir_0):
    # Load and preprocess the image
    img = load_img(os.path.join(dataset_dir_0, image_file), target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    # Append the image to the images list
    images_patient_13.append(img_array)
    # Extract the label from the image filename
    label = os.path.splitext(image_file)[0]  # Remove file extension
    ids_patient_13.append(label)
    # Assuming you also have a list called labels, you need to define it somewhere in your code
    # labels.append(0)

# Now ids_patient_1 contains the names of the images


Number of image files in the directory for label 0: 12053


In [28]:
len(images_patient_13)

12053

In [29]:

# Define the paths to your dataset directories
dataset_dir_0 = "patient_14"

# Initialize empty lists to store images and labels
# Define image dimensions
img_height, img_width = 100, 100
images_patient_14 = []
ids_patient_14 = []  # List to store image names
# Load and preprocess images from directory 0 (label 0)
image_files = os.listdir(dataset_dir_0)
print("Number of image files in the directory for label 0:", len(image_files))

for image_file in os.listdir(dataset_dir_0):
    # Load and preprocess the image
    img = load_img(os.path.join(dataset_dir_0, image_file), target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    # Append the image to the images list
    images_patient_14.append(img_array)
    # Extract the label from the image filename
    label = os.path.splitext(image_file)[0]  # Remove file extension
    ids_patient_14.append(label)
    # Assuming you also have a list called labels, you need to define it somewhere in your code
    # labels.append(0)

# Now ids_patient_1 contains the names of the images


Number of image files in the directory for label 0: 16327


In [30]:
len(images_patient_14)

16327

In [31]:

# Define the paths to your dataset directories
dataset_dir_0 = "patient_15"

# Initialize empty lists to store images and labels
# Define image dimensions
img_height, img_width = 100, 100
images_patient_15 = []
ids_patient_15 = []  # List to store image names
# Load and preprocess images from directory 0 (label 0)
image_files = os.listdir(dataset_dir_0)
print("Number of image files in the directory for label 0:", len(image_files))

for image_file in os.listdir(dataset_dir_0):
    # Load and preprocess the image
    img = load_img(os.path.join(dataset_dir_0, image_file), target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    # Append the image to the images list
    images_patient_15.append(img_array)
    # Extract the label from the image filename
    label = os.path.splitext(image_file)[0]  # Remove file extension
    ids_patient_15.append(label)
    # Assuming you also have a list called labels, you need to define it somewhere in your code
    # labels.append(0)

# Now ids_patient_1 contains the names of the images


Number of image files in the directory for label 0: 13487


In [32]:
len(images_patient_15)

13487

## Healthy

In [33]:

# Define the paths to your dataset directories
dataset_dir_0 = "healthy_11"

# Initialize empty lists to store images and labels
# Define image dimensions
img_height, img_width = 100, 100
images_healthy_11 = []
ids_healthy_11 = []  # List to store image names
# Load and preprocess images from directory 0 (label 0)
image_files = os.listdir(dataset_dir_0)
print("Number of image files in the directory for label 0:", len(image_files))

for image_file in os.listdir(dataset_dir_0):
    # Load and preprocess the image
    img = load_img(os.path.join(dataset_dir_0, image_file), target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    # Append the image to the images list
    images_healthy_11.append(img_array)
    # Extract the label from the image filename
    label = os.path.splitext(image_file)[0]  # Remove file extension
    ids_healthy_11.append(label)

    # Assuming you also have a list called labels, you need to define it somewhere in your code
    # labels.append(0)

# Now ids_patient_1 contains the names of the images


Number of image files in the directory for label 0: 14512


In [34]:
len(images_healthy_11)

14512

In [35]:

# Define the paths to your dataset directories
dataset_dir_0 = "healthy_12"

# Initialize empty lists to store images and labels
# Define image dimensions
img_height, img_width = 100, 100
images_healthy_12 = []
ids_healthy_12 = []  # List to store image names
# Load and preprocess images from directory 0 (label 0)
image_files = os.listdir(dataset_dir_0)
print("Number of image files in the directory for label 0:", len(image_files))

for image_file in os.listdir(dataset_dir_0):
    # Load and preprocess the image
    img = load_img(os.path.join(dataset_dir_0, image_file), target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    # Append the image to the images list
    images_healthy_12.append(img_array)
    # Extract the label from the image filename
    label = os.path.splitext(image_file)[0]  # Remove file extension
    ids_healthy_12.append(label)
# Now ids_patient_1 contains the names of the images


Number of image files in the directory for label 0: 7162


In [36]:
len(images_healthy_12)

7162

In [37]:

# Define the paths to your dataset directories
dataset_dir_0 = "healthy_13"

# Initialize empty lists to store images and labels
# Define image dimensions
img_height, img_width = 100, 100
images_healthy_13 = []
ids_healthy_13 = []  # List to store image names
# Load and preprocess images from directory 0 (label 0)
image_files = os.listdir(dataset_dir_0)
print("Number of image files in the directory for label 0:", len(image_files))

for image_file in os.listdir(dataset_dir_0):
    # Load and preprocess the image
    img = load_img(os.path.join(dataset_dir_0, image_file), target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    # Append the image to the images list
    images_healthy_13.append(img_array)
    # Extract the label from the image filename
    label = os.path.splitext(image_file)[0]  # Remove file extension
    ids_healthy_13.append(label)

# Now ids_patient_1 contains the names of the images


Number of image files in the directory for label 0: 12271


In [38]:
len(images_healthy_13)

12271

In [39]:

# Define the paths to your dataset directories
dataset_dir_0 = "healthy_14"

# Initialize empty lists to store images and labels
# Define image dimensions
img_height, img_width = 100, 100
images_healthy_14 = []
ids_healthy_14 = []  # List to store image names
# Load and preprocess images from directory 0 (label 0)
image_files = os.listdir(dataset_dir_0)
print("Number of image files in the directory for label 0:", len(image_files))

for image_file in os.listdir(dataset_dir_0):
    # Load and preprocess the image
    img = load_img(os.path.join(dataset_dir_0, image_file), target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    # Append the image to the images list
    images_healthy_14.append(img_array)
    # Extract the label from the image filename
    label = os.path.splitext(image_file)[0]  # Remove file extension
    ids_healthy_14.append(label)

    # Assuming you also have a list called labels, you need to define it somewhere in your code
    # labels.append(0)

# Now ids_patient_1 contains the names of the images


Number of image files in the directory for label 0: 10941


In [40]:
len(images_healthy_14)

10941

In [41]:

# Define the paths to your dataset directories
dataset_dir_0 = "healthy_15"

# Initialize empty lists to store images and labels
# Define image dimensions
img_height, img_width = 100, 100
images_healthy_15 = []
ids_healthy_15 = []  # List to store image names
# Load and preprocess images from directory 0 (label 0)
image_files = os.listdir(dataset_dir_0)
print("Number of image files in the directory for label 0:", len(image_files))

for image_file in os.listdir(dataset_dir_0):
    # Load and preprocess the image
    img = load_img(os.path.join(dataset_dir_0, image_file), target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    # Append the image to the images list
    images_healthy_15.append(img_array)
    # Extract the label from the image filename
    label = os.path.splitext(image_file)[0]  # Remove file extension
    ids_healthy_15.append(label)
# Now ids_patient_1 contains the names of the images


Number of image files in the directory for label 0: 10741


In [42]:
len(images_healthy_15)

10741

In [43]:

# Define the paths to your dataset directories
dataset_dir_0 = "healthy_16"

# Initialize empty lists to store images and labels
# Define image dimensions
img_height, img_width = 100, 100
images_healthy_16 = []
ids_healthy_16 = []  # List to store image names
# Load and preprocess images from directory 0 (label 0)
image_files = os.listdir(dataset_dir_0)
print("Number of image files in the directory for label 0:", len(image_files))

for image_file in os.listdir(dataset_dir_0):
    # Load and preprocess the image
    img = load_img(os.path.join(dataset_dir_0, image_file), target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    # Append the image to the images list
    images_healthy_16.append(img_array)
    # Extract the label from the image filename
    label = os.path.splitext(image_file)[0]  # Remove file extension
    ids_healthy_16.append(label)

# Now ids_patient_1 contains the names of the images


Number of image files in the directory for label 0: 4965


In [44]:
len(images_healthy_16)

4965

In [45]:

# Define the paths to your dataset directories
dataset_dir_0 = "healthy_17"

# Initialize empty lists to store images and labels
# Define image dimensions
img_height, img_width = 100, 100
images_healthy_17 = []
ids_healthy_17 = []  # List to store image names
# Load and preprocess images from directory 0 (label 0)
image_files = os.listdir(dataset_dir_0)
print("Number of image files in the directory for label 0:", len(image_files))

for image_file in os.listdir(dataset_dir_0):
    # Load and preprocess the image
    img = load_img(os.path.join(dataset_dir_0, image_file), target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    # Append the image to the images list
    images_healthy_17.append(img_array)
    # Extract the label from the image filename
    label = os.path.splitext(image_file)[0]  # Remove file extension
    ids_healthy_17.append(label)

    # Assuming you also have a list called labels, you need to define it somewhere in your code
    # labels.append(0)

# Now ids_patient_1 contains the names of the images


Number of image files in the directory for label 0: 6671


In [46]:
len(images_healthy_17)

6671

In [47]:

# Define the paths to your dataset directories
dataset_dir_0 = "healthy_18"

# Initialize empty lists to store images and labels
# Define image dimensions
img_height, img_width = 100, 100
images_healthy_18 = []
ids_healthy_18 = []  # List to store image names
# Load and preprocess images from directory 0 (label 0)
image_files = os.listdir(dataset_dir_0)
print("Number of image files in the directory for label 0:", len(image_files))

for image_file in os.listdir(dataset_dir_0):
    # Load and preprocess the image
    img = load_img(os.path.join(dataset_dir_0, image_file), target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    # Append the image to the images list
    images_healthy_18.append(img_array)
    # Extract the label from the image filename
    label = os.path.splitext(image_file)[0]  # Remove file extension
    ids_healthy_18.append(label)
# Now ids_patient_1 contains the names of the images


Number of image files in the directory for label 0: 9216


In [48]:
len(images_healthy_18)

9216

In [49]:

# Define the paths to your dataset directories
dataset_dir_0 = "healthy_19"

# Initialize empty lists to store images and labels
# Define image dimensions
img_height, img_width = 100, 100
images_healthy_19 = []
ids_healthy_19 = []  # List to store image names
# Load and preprocess images from directory 0 (label 0)
image_files = os.listdir(dataset_dir_0)
print("Number of image files in the directory for label 0:", len(image_files))

for image_file in os.listdir(dataset_dir_0):
    # Load and preprocess the image
    img = load_img(os.path.join(dataset_dir_0, image_file), target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    # Append the image to the images list
    images_healthy_19.append(img_array)
    # Extract the label from the image filename
    label = os.path.splitext(image_file)[0]  # Remove file extension
    ids_healthy_19.append(label)
# Now ids_patient_1 contains the names of the images


Number of image files in the directory for label 0: 14460


In [50]:
len(images_healthy_19)

14460

In [51]:

# Define the paths to your dataset directories
dataset_dir_0 = "healthy_20"

# Initialize empty lists to store images and labels
# Define image dimensions
img_height, img_width = 100, 100
images_healthy_20 = []
ids_healthy_20 = []  # List to store image names
# Load and preprocess images from directory 0 (label 0)
image_files = os.listdir(dataset_dir_0)
print("Number of image files in the directory for label 0:", len(image_files))

for image_file in os.listdir(dataset_dir_0):
    # Load and preprocess the image
    img = load_img(os.path.join(dataset_dir_0, image_file), target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    # Append the image to the images list
    images_healthy_20.append(img_array)
    # Extract the label from the image filename
    label = os.path.splitext(image_file)[0]  # Remove file extension
    ids_healthy_20.append(label)
# Now ids_patient_1 contains the names of the images


Number of image files in the directory for label 0: 14893


In [52]:
len(images_healthy_20)

14893

#  Model Generic PATIENTS 1

In [53]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('generic_model_ARGvshealthy_and_patient_synthetic_1.h5')

# Now you can use this 'model' object for predictions or further training.


## PATIENTS

### Patient 1

In [54]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_1)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_1))


344/344 [==============================] - 16s 41ms/step
Number of predictions over 0.5: 1494
Number of total images:  10981


### Patient 2

In [55]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_2)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_2))


469/469 [==============================] - 19s 41ms/step
Number of predictions over 0.5: 2894
Number of total images:  15000


### Patient 4

In [56]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_4)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_4))


287/287 [==============================] - 10s 34ms/step
Number of predictions over 0.5: 184
Number of total images:  9172


### Patient 5

In [57]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_5)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_5))


469/469 [==============================] - 18s 37ms/step
Number of predictions over 0.5: 431
Number of total images:  15000


### Patient 6

In [58]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_6)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_6))


321/321 [==============================] - 11s 33ms/step
Number of predictions over 0.5: 577
Number of total images:  10267


### Patient 11

In [59]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_11)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_11))


314/314 [==============================] - 12s 37ms/step
Number of predictions over 0.5: 1149
Number of total images:  10022


### Patient 12

In [60]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_12)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_12))


264/264 [==============================] - 9s 33ms/step
Number of predictions over 0.5: 829
Number of total images:  8445


### Patient 13

In [61]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_13)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_13))


377/377 [==============================] - 14s 37ms/step
Number of predictions over 0.5: 1100
Number of total images:  12053


### Patient 14

In [62]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_14)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_14))


511/511 [==============================] - 20s 38ms/step
Number of predictions over 0.5: 1221
Number of total images:  16327


### Patient 15

In [63]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_15)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_15))


422/422 [==============================] - 14s 34ms/step
Number of predictions over 0.5: 1454
Number of total images:  13487


## HEALTHIES

### Healthy 11

In [64]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_11)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_11))

454/454 [==============================] - 19s 42ms/step
Number of predictions over 0.5: 1820
Number of total images:  14512


### Healthy 12

In [65]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_12)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_12))

224/224 [==============================] - 8s 34ms/step
Number of predictions over 0.5: 781
Number of total images:  7162


### Healthy 13

In [66]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_13)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_13))

384/384 [==============================] - 15s 39ms/step
Number of predictions over 0.5: 1436
Number of total images:  12271


### Healthy 14

In [67]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_14)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_14))

342/342 [==============================] - 11s 33ms/step
Number of predictions over 0.5: 226
Number of total images:  10941


### Healthy 15

In [68]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_15)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_15))

336/336 [==============================] - 12s 35ms/step
Number of predictions over 0.5: 191
Number of total images:  10741


### Healthy 16

In [69]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_16)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_16))

156/156 [==============================] - 5s 34ms/step
Number of predictions over 0.5: 247
Number of total images:  4965


### Healthy 17

In [70]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_17)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_17))

209/209 [==============================] - 7s 35ms/step
Number of predictions over 0.5: 459
Number of total images:  6671


### Healthy 18

In [71]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_18)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_18))

288/288 [==============================] - 12s 40ms/step
Number of predictions over 0.5: 617
Number of total images:  9216


### Healthy 19

In [72]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_19)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_19))

452/452 [==============================] - 17s 37ms/step
Number of predictions over 0.5: 2165
Number of total images:  14460


### Healthy 20

In [73]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_20)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_20))

466/466 [==============================] - 17s 37ms/step
Number of predictions over 0.5: 2148
Number of total images:  14893


#  Model Generic PATIENTS 2

In [74]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('generic_model_ARGvshealthy_and_patient_synthetic_2.h5')

# Now you can use this 'model' object for predictions or further training.


## PATIENTS

### Patient 1

In [75]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_1)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_1))


344/344 [==============================] - 12s 34ms/step
Number of predictions over 0.5: 1417
Number of total images:  10981


### Patient 2

In [76]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_2)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_2))


469/469 [==============================] - 16s 34ms/step
Number of predictions over 0.5: 5341
Number of total images:  15000


### Patient 4

In [77]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_4)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_4))


287/287 [==============================] - 9s 32ms/step
Number of predictions over 0.5: 2150
Number of total images:  9172


### Patient 5

In [78]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_5)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_5))


469/469 [==============================] - 17s 35ms/step
Number of predictions over 0.5: 3839
Number of total images:  15000


### Patient 6

In [79]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_6)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_6))


321/321 [==============================] - 11s 33ms/step
Number of predictions over 0.5: 1030
Number of total images:  10267


### Patient 11

In [80]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_11)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_11))


314/314 [==============================] - 10s 33ms/step
Number of predictions over 0.5: 2462
Number of total images:  10022


### Patient 12

In [81]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_12)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_12))


264/264 [==============================] - 9s 36ms/step
Number of predictions over 0.5: 2395
Number of total images:  8445


### Patient 13

In [82]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_13)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_13))


377/377 [==============================] - 14s 38ms/step
Number of predictions over 0.5: 2897
Number of total images:  12053


### Patient 14

In [83]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_14)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_14))


511/511 [==============================] - 19s 38ms/step
Number of predictions over 0.5: 4123
Number of total images:  16327


### Patient 15

In [84]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_15)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_15))


422/422 [==============================] - 15s 34ms/step
Number of predictions over 0.5: 3965
Number of total images:  13487


## HEALTHIES

### Healthy 11

In [85]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_11)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_11))

454/454 [==============================] - 18s 40ms/step
Number of predictions over 0.5: 4692
Number of total images:  14512


### Healthy 12

In [86]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_12)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_12))

224/224 [==============================] - 8s 34ms/step
Number of predictions over 0.5: 1083
Number of total images:  7162


### Healthy 13

In [87]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_13)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_13))

384/384 [==============================] - 15s 39ms/step
Number of predictions over 0.5: 3639
Number of total images:  12271


### Healthy 14

In [88]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_14)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_14))

342/342 [==============================] - 12s 35ms/step
Number of predictions over 0.5: 2200
Number of total images:  10941


### Healthy 15

In [89]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_15)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_15))

336/336 [==============================] - 13s 39ms/step
Number of predictions over 0.5: 2781
Number of total images:  10741


### Healthy 16

In [90]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_16)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_16))

156/156 [==============================] - 6s 37ms/step
Number of predictions over 0.5: 619
Number of total images:  4965


### Healthy 17

In [91]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_17)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_17))

209/209 [==============================] - 7s 32ms/step
Number of predictions over 0.5: 573
Number of total images:  6671


### Healthy 18

In [92]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_18)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_18))

288/288 [==============================] - 10s 34ms/step
Number of predictions over 0.5: 2055
Number of total images:  9216


### Healthy 19

In [93]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_19)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_19))

452/452 [==============================] - 18s 39ms/step
Number of predictions over 0.5: 4752
Number of total images:  14460


### Healthy 20

In [94]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_20)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_20))

466/466 [==============================] - 17s 35ms/step
Number of predictions over 0.5: 4852
Number of total images:  14893


#  Model Generic PATIENT 3

In [95]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('generic_model_ARGvshealthy_and_patient_synthetic_3.h5')

# Now you can use this 'model' object for predictions or further training.


## PATIENTS

### Patient 1

In [96]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_1)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_1))


344/344 [==============================] - 12s 33ms/step
Number of predictions over 0.5: 1988
Number of total images:  10981


### Patient 2

In [97]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_2)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_2))


469/469 [==============================] - 16s 34ms/step
Number of predictions over 0.5: 1520
Number of total images:  15000


### Patient 4

In [98]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_4)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_4))


287/287 [==============================] - 10s 34ms/step
Number of predictions over 0.5: 1849
Number of total images:  9172


### Patient 5

In [99]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_5)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_5))


469/469 [==============================] - 17s 37ms/step
Number of predictions over 0.5: 2504
Number of total images:  15000


### Patient 6

In [100]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_6)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_6))


321/321 [==============================] - 11s 34ms/step
Number of predictions over 0.5: 610
Number of total images:  10267


### Patient 11

In [101]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_11)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_11))


314/314 [==============================] - 11s 36ms/step
Number of predictions over 0.5: 1891
Number of total images:  10022


### Patient 12

In [102]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_12)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_12))


264/264 [==============================] - 9s 35ms/step
Number of predictions over 0.5: 1450
Number of total images:  8445


### Patient 13

In [103]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_13)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_13))


377/377 [==============================] - 14s 37ms/step
Number of predictions over 0.5: 2222
Number of total images:  12053


### Patient 14

In [104]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_14)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_14))


511/511 [==============================] - 19s 37ms/step
Number of predictions over 0.5: 3914
Number of total images:  16327


### Patient 15

In [105]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_patient_15)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_patient_15))


422/422 [==============================] - 14s 34ms/step
Number of predictions over 0.5: 2711
Number of total images:  13487


## HEALTHIES

### Healthy 11

In [106]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_11)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_11))

454/454 [==============================] - 16s 36ms/step
Number of predictions over 0.5: 2382
Number of total images:  14512


### Healthy 12

In [107]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_12)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_12))

224/224 [==============================] - 7s 33ms/step
Number of predictions over 0.5: 889
Number of total images:  7162


### Healthy 13

In [108]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_13)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_13))

384/384 [==============================] - 15s 38ms/step
Number of predictions over 0.5: 1713
Number of total images:  12271


### Healthy 14

In [109]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_14)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_14))

342/342 [==============================] - 12s 36ms/step
Number of predictions over 0.5: 1684
Number of total images:  10941


### Healthy 15

In [110]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_15)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_15))

336/336 [==============================] - 12s 35ms/step
Number of predictions over 0.5: 1481
Number of total images:  10741


### Healthy 16

In [111]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_16)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_16))

156/156 [==============================] - 5s 33ms/step
Number of predictions over 0.5: 321
Number of total images:  4965


### Healthy 17

In [112]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_17)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_17))

209/209 [==============================] - 8s 37ms/step
Number of predictions over 0.5: 904
Number of total images:  6671


### Healthy 18

In [113]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_18)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_18))

288/288 [==============================] - 9s 33ms/step
Number of predictions over 0.5: 319
Number of total images:  9216


### Healthy 19

In [114]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_19)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_19))

452/452 [==============================] - 17s 38ms/step
Number of predictions over 0.5: 2300
Number of total images:  14460


### Healthy 20

In [115]:
# Assuming 'model' contains the loaded model and 'images_patient_1' contains the preprocessed images
import numpy as np

# Convert the list of images to a numpy array
images_array = np.array(images_healthy_20)

# Make predictions on the images
predictions = model.predict(images_array)

# Filter predictions with values over 0.5
threshold = 0.5
predicted_labels_over_threshold = np.where(predictions > threshold)[0]

# Count the number of predictions over the threshold
num_predictions_over_threshold = len(predicted_labels_over_threshold)

# Print the length of predicted labels over 0.5
print("Number of predictions over 0.5:", num_predictions_over_threshold)
print("Number of total images: ",len(images_healthy_20))

466/466 [==============================] - 17s 37ms/step
Number of predictions over 0.5: 3565
Number of total images:  14893
